In [1]:
import VGG as vgg
import torch
from HSI_class import HSI
import numpy as np
from tqdm import tqdm

import os

directory = r"C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune"
files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# directory2 = r"C:\Users\Asus TUF\Documents\code\TA\models\model_800train"
# files2 = [os.path.join(directory2, f) for f in os.listdir(directory2) if os.path.isfile(os.path.join(directory2, f))]


models = []

for file in files:
    models.append(file)

# for file in files2:
#     models.append(file)

for model in models:
    print(model)


C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0005.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0010.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0020.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0030.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0040.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0050.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0060.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0070.pth.tar
C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\finetune\20250505_005434_checkpoint_0080.pth.tar
C:\Users\Asus TUF\Documents\

In [2]:
dataset_path = "Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 11:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

Processing file: Hyperspectral oil spill detection datasets\GM01.mat
Processing file: Hyperspectral oil spill detection datasets\GM02.mat
Processing file: Hyperspectral oil spill detection datasets\GM03.mat
Processing file: Hyperspectral oil spill detection datasets\GM04.mat
Processing file: Hyperspectral oil spill detection datasets\GM05.mat
Processing file: Hyperspectral oil spill detection datasets\GM06.mat
Processing file: Hyperspectral oil spill detection datasets\GM07.mat
Processing file: Hyperspectral oil spill detection datasets\GM08.mat
Processing file: Hyperspectral oil spill detection datasets\GM09.mat
Processing file: Hyperspectral oil spill detection datasets\GM10.mat
Processing file: Hyperspectral oil spill detection datasets\GM11.mat
Processing file: Hyperspectral oil spill detection datasets\GM12.mat


In [15]:
import zeroPadding
hsi_test = dataset[11]

test_img = hsi_test.img
test_gt = hsi_test.gt

patch_size = 9
half_patch = patch_size // 2

height = test_img.shape[0]
width = test_img.shape[1]

matrix=zeroPadding.zeroPadding_3D(test_img,half_patch) #add 0 in every side of the data
print(f"img shape: {test_img.shape}")
print(f"img shape after padding {matrix.shape}")
print(f"number of pixel {width * height}")

img shape: (842, 640, 224)
img shape after padding (850, 648, 224)
number of pixel 538880


In [16]:
print(test_gt.shape)

indices0 = np.argwhere(test_gt == 0)
indices1 = np.argwhere(test_gt == 1)

print(indices0.shape)
print(indices1.shape)

num_samples = 1000

random_indices0 = indices0[np.random.choice(len(indices0), num_samples, replace=False)]
random_indices1 = indices1[np.random.choice(len(indices1), num_samples, replace=False)]

test_indices = np.vstack((random_indices0, random_indices1))

print(test_indices.shape)

(842, 640)
(521713, 2)
(17167, 2)
(2000, 2)


In [17]:

def predict_batch(model, batch_input, device):
    model.eval()
    batch_input = batch_input.to(device)

    with torch.no_grad():
        output = model(batch_input)

    predicted_classes = torch.argmax(output, dim=1).cpu().numpy()
    confidences = torch.nn.functional.softmax(output, dim=1)
    confidences = confidences[range(len(predicted_classes)), predicted_classes].cpu().numpy()

    return predicted_classes, confidences

In [18]:
from torchvision.models import vgg16
import torch.nn as nn

class VGG16_HSI(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16_HSI, self).__init__()

         # Custom Convolutional Layer: Process 9x9x224 input
        self.pre_conv = nn.Sequential(
            nn.Conv2d(in_channels=224, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AdaptiveAvgPool2d((1, 1))  # Reduce to (256, 1, 1)
        )

        # Fully Connected Layer to reshape to (64, 56, 56)
        self.fc = nn.Linear(256 * 1 * 1, 64 * 56 * 56)

        # Load VGG-16 Model
        self.encoder = vgg16(pretrained=False)

        # Remove first VGG-16 conv layer
        self.encoder.features = nn.Sequential(*list(self.encoder.features.children())[1:])

        # Modify classifier to output 2 classes
        self.encoder.classifier[6] = nn.Linear(4096, 2)

    def forward(self, x):
        # print(f'before {x.shape}')
        x = self.pre_conv(x)  # Process hyperspectral input
        x = x.view(x.size(0), -1)  # Flatten

        # print(f'after preconv {x.shape}')
        x = self.fc(x)  # Fully connected layer
        # print(f'after fc {x.shape}')
        # Reshape to (batch_size, 64, 56, 56) before passing to VGG
        x = x.view(x.size(0), 64, 56, 56)
        # print(f'after reshape, before vgg second layer {x.shape}')

        x = self.encoder.features(x)  # Pass to VGG-16
        x = self.encoder.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.encoder.classifier(x)  # Final classification layer

        return x

In [19]:
# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64  # You can change this depending on your GPU capacity
scores = []

for model_path in models:
    model_name = model_path.split('\\')[-1]
    print(f"Creating model {model_name}...")
    saved_model = VGG16_HSI().to(device)
    checkpoint = torch.load(model_path, map_location=device)
    saved_model.load_state_dict(checkpoint['state_dict'])
    print("Model loaded and moved to device")

    total = len(test_indices)
    correct0 = 0
    correct1 = 0

    input_patches = []
    true_labels = []

    # Prepare all patches
    for x_pos, y_pos in test_indices:
        true_label = test_gt[x_pos][y_pos]

        selected_rows = matrix[x_pos:x_pos + 2*half_patch + 1, :]
        testing_patch = selected_rows[:, y_pos:y_pos + 2*half_patch + 1]

        patch_tensor = torch.tensor(testing_patch, dtype=torch.float32)
        patch_tensor = patch_tensor.unsqueeze(0).permute(0, 3, 1, 2)

        input_patches.append(patch_tensor)
        true_labels.append(true_label)

    input_patches = torch.cat(input_patches, dim=0)  # Shape: (N, C, H, W)
    true_labels = torch.tensor(true_labels)

    # Process in batches
    for i in tqdm(range(0, total, batch_size), desc="Predicting"):
        batch = input_patches[i:i+batch_size]
        labels = true_labels[i:i+batch_size]

        preds, confs = predict_batch(saved_model, batch, device)

        for j in range(len(preds)):
            index = i + j
            # print(f"{index+1}: prediction = {preds[j]}, confidence: {confs[j]:.4f}, expected: {labels[j].item()}")
            if preds[j] == labels[j].item():
                if labels[j].item() == 0:
                    correct0 += 1
                elif labels[j] == 1:
                    correct1 += 1

    correct = correct0 + correct1
    print(f"Score: {correct}/{total}")
    
    scores.append((model_name, f'{correct0}/{total/2}', f'{correct1}/{total/2}', f'{correct}/{total}'))

Creating model 20250505_005434_checkpoint_0005.pth.tar...


c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Asus TUF\AppData\Local\Temp\ipykernel_3048\2945208158.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default va

Model loaded and moved to device


Predicting: 100%|██████████| 32/32 [00:02<00:00, 15.33it/s]


Score: 1000/2000
Creating model 20250505_005434_checkpoint_0010.pth.tar...
Model loaded and moved to device


...

Score: 1026/2000
Creating model 20250505_005831_checkpoint_0175.pth.tar...
Model loaded and moved to device


Predicting: 100%|██████████| 32/32 [00:02<00:00, 15.35it/s]


Score: 1310/2000
Creating model 20250505_005831_checkpoint_0200.pth.tar...
Model loaded and moved to device


Predicting: 100%|██████████| 32/32 [00:02<00:00, 15.78it/s]

Score: 1068/2000


In [20]:
# print(total)
for score in scores:
    print(score)

('20250505_005434_checkpoint_0005.pth.tar', '7/1000.0', '993/1000.0', '1000/2000')
('20250505_005434_checkpoint_0010.pth.tar', '10/1000.0', '992/1000.0', '1002/2000')
('20250505_005434_checkpoint_0020.pth.tar', '642/1000.0', '932/1000.0', '1574/2000')
('20250505_005434_checkpoint_0030.pth.tar', '837/1000.0', '837/1000.0', '1674/2000')
('20250505_005434_checkpoint_0040.pth.tar', '494/1000.0', '788/1000.0', '1282/2000')
('20250505_005434_checkpoint_0050.pth.tar', '513/1000.0', '258/1000.0', '771/2000')
('20250505_005434_checkpoint_0060.pth.tar', '548/1000.0', '504/1000.0', '1052/2000')
('20250505_005434_checkpoint_0070.pth.tar', '999/1000.0', '12/1000.0', '1011/2000')
('20250505_005434_checkpoint_0080.pth.tar', '638/1000.0', '933/1000.0', '1571/2000')
('20250505_005434_checkpoint_0090.pth.tar', '856/1000.0', '791/1000.0', '1647/2000')
('20250505_005434_checkpoint_0100.pth.tar', '651/1000.0', '811/1000.0', '1462/2000')
('20250505_005434_checkpoint_0125.pth.tar', '497/1000.0', '679/1000.0'